<a href="https://colab.research.google.com/github/Jazzystic/data_code_migration_GFG/blob/main/migraci%C3%B3n_hist%C3%B3rico_comentarios_GFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd

# Subir archivo
print("Selecciona el archivo comentarios.csv:")
uploaded = files.upload()

# Verificar que se subió
print("\nArchivo(s) subido(s):")
for filename in uploaded.keys():
    print(f"- {filename} ({len(uploaded[filename])} bytes)")

Selecciona el archivo comentarios.csv:


Saving comentarios.csv to comentarios (1).csv

Archivo(s) subido(s):
- comentarios (1).csv (2879435 bytes)


In [2]:
# Leer el CSV original
df = pd.read_csv('comentarios.csv',
                 encoding='utf-8',
                 on_bad_lines='skip')

# Información básica
print("Primeras 5 filas:")
print(df.head())
print("\n" + "="*50)
print("Columnas detectadas:")
print(df.columns.tolist())
print("\n" + "="*50)
print(f"Total de filas: {len(df)}")
print("\n" + "="*50)
print("Valores nulos por columna:")
print(df.isnull().sum())

Primeras 5 filas:
                               hotel_name date_posted date_response  \
0  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
1  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
2  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
3               The St. Regis Mexico City  2019-12-01    2019-12-25   
4               The St. Regis Mexico City  2019-12-01    2019-12-25   

       channel    rating                                       title_review  \
0  TripAdvisor   Average  Atencion del restaurante y calidad de la habit...   
1  TripAdvisor   Average  Atencion del restaurante y calidad de la habit...   
2  TripAdvisor   Average  Atencion del restaurante y calidad de la habit...   
3  TripAdvisor  Terrible  This St Regis Is not a 5* property any longer,...   
4  TripAdvisor  Terrible  This St Regis Is not a 5* property any longer,...   

                                        comment_text     key_reason  
0  F&B foo

In [3]:
# PASO 3: Limpiar y exportar
df_clean = df.copy()

# 1. Limpiar campos de texto (quitar saltos de línea internos)
text_columns = ['title_review', 'comment_text', 'key_reason']
for col in text_columns:
    if col in df_clean.columns:
        # Convertir a string y limpiar saltos de línea
        df_clean[col] = df_clean[col].fillna('').astype(str).str.replace('\n', ' ', regex=False).str.replace('\r', ' ', regex=False)

# 2. Convertir fechas al formato PostgreSQL (YYYY-MM-DD)
date_columns = ['date_posted', 'date_response']
for col in date_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce').dt.strftime('%Y-%m-%d')

# 3. Reemplazar 'NaT' (fechas inválidas) por NULL
df_clean = df_clean.replace({'NaT': None})

# 4. Exportar CSV limpio con comillas en TODOS los campos
df_clean.to_csv('comentarios_clean.csv',
                index=False,
                encoding='utf-8',
                quoting=1,  # QUOTE_ALL
                escapechar='\\',
                doublequote=True)

print("✅ Archivo limpio generado: comentarios_clean.csv")
print(f"📊 Filas exportadas: {len(df_clean)}")
print("\n🔍 Muestra de filas limpias:")
print(df_clean.head(3))

# Descargar archivo limpio
from google.colab import files
files.download('comentarios_clean.csv')

✅ Archivo limpio generado: comentarios_clean.csv
📊 Filas exportadas: 15631

🔍 Muestra de filas limpias:
                               hotel_name date_posted date_response  \
0  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
1  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
2  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   

       channel   rating                                       title_review  \
0  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   
1  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   
2  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   

                                        comment_text key_reason  
0  F&B food is okey but the attitude of the staff...        F&B  
1  Room air condition sound is to loud therefore ...       Room  
2     Room bed is very hard and not very comfortable       Room  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
# PASO 3 MEJORADO: Limpiar y exportar con NULL explícito
df_clean = df.copy()

# 1. Limpiar campos de texto (quitar saltos de línea internos)
text_columns = ['title_review', 'comment_text', 'key_reason']
for col in text_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna('').astype(str).str.replace('\n', ' ', regex=False).str.replace('\r', ' ', regex=False)

# 2. Convertir fechas al formato PostgreSQL (YYYY-MM-DD)
date_columns = ['date_posted', 'date_response']
for col in date_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
        # Convertir NaT a None (NULL en SQL) ANTES de formatear
        df_clean[col] = df_clean[col].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notna(x) else None)

# 3. Limpiar otros campos: convertir strings vacíos a None
for col in ['hotel_name', 'channel', 'rating', 'title_review', 'comment_text', 'key_reason']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].replace('', None)

# 4. Exportar CSV limpio
df_clean.to_csv('comentarios_clean.csv',
                index=False,
                encoding='utf-8',
                quoting=1,  # QUOTE_ALL
                escapechar='\\',
                doublequote=True,
                na_rep='')  # Representar NULL como vacío (pgAdmin lo interpreta como NULL)

print("✅ Archivo limpio generado: comentarios_clean.csv")
print(f"📊 Filas exportadas: {len(df_clean)}")
print("\n🔍 Muestra de línea 107 (la problemática):")
print(df_clean.iloc[106:108])  # Mostrar líneas 107-108 (índice 106-107)

# Descargar archivo limpio
from google.colab import files
files.download('comentarios_clean.csv')

✅ Archivo limpio generado: comentarios_clean.csv
📊 Filas exportadas: 15631

🔍 Muestra de línea 107 (la problemática):
                                hotel_name date_posted date_response  \
106  Dreams Vista Cancun Golf & Spa Resort  2021-11-29    2021-11-30   
107    Dreams Bahia Mita Surf & Spa Resort  2021-11-24    2021-11-24   

         channel   rating               title_review  \
106  TripAdvisor  Average                   Nice…but   
107  TripAdvisor  Average  Left feeling underwhelmed   

                                          comment_text    key_reason  
106                      Beach there was broken glass.         Beach  
107  Distribution Felt excluded due to half of the ...  Distribution  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
# PASO 3 DEFINITIVO: Usar \N para NULL
df_clean = df.copy()

# 1. Limpiar campos de texto
text_columns = ['title_review', 'comment_text', 'key_reason']
for col in text_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna('').astype(str).str.replace('\n', ' ', regex=False).str.replace('\r', ' ', regex=False)
        df_clean[col] = df_clean[col].replace('', '\\N')  # Vacíos = \N

# 2. Convertir fechas
date_columns = ['date_posted', 'date_response']
for col in date_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
        df_clean[col] = df_clean[col].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notna(x) else '\\N')

# 3. Otros campos vacíos a \N
for col in ['hotel_name', 'channel', 'rating', 'key_reason']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna('\\N').replace('', '\\N')

# 4. Exportar
df_clean.to_csv('comentarios_clean.csv',
                index=False,
                encoding='utf-8',
                quoting=1,
                escapechar='\\',
                doublequote=True)

print("✅ Archivo con \\N para NULL generado")
print(f"📊 Filas: {len(df_clean)}")
print("\n🔍 Línea 107:")
print(df_clean.iloc[106:108])

from google.colab import files
files.download('comentarios_clean.csv')

✅ Archivo con \N para NULL generado
📊 Filas: 15631

🔍 Línea 107:
                                hotel_name date_posted date_response  \
106  Dreams Vista Cancun Golf & Spa Resort  2021-11-29    2021-11-30   
107    Dreams Bahia Mita Surf & Spa Resort  2021-11-24    2021-11-24   

         channel   rating               title_review  \
106  TripAdvisor  Average                   Nice…but   
107  TripAdvisor  Average  Left feeling underwhelmed   

                                          comment_text    key_reason  
106                      Beach there was broken glass.         Beach  
107  Distribution Felt excluded due to half of the ...  Distribution  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# PASO 3 FINAL: Vacíos sin comillas para que sean NULL
df_clean = df.copy()

# 1. Limpiar campos de texto (mantener strings, vacíos los dejamos como NaN)
text_columns = ['title_review', 'comment_text', 'key_reason']
for col in text_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str)
        df_clean[col] = df_clean[col].replace('nan', '')  # pandas convierte NaN a 'nan'
        df_clean[col] = df_clean[col].str.replace('\n', ' ', regex=False)
        df_clean[col] = df_clean[col].str.replace('\r', ' ', regex=False)
        df_clean[col] = df_clean[col].replace('', None)  # Vacíos a None

# 2. Convertir fechas (dejar None para vacíos)
date_columns = ['date_posted', 'date_response']
for col in date_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
        df_clean[col] = df_clean[col].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notna(x) else None)

# 3. Otros campos: vacíos a None
for col in ['hotel_name', 'channel', 'rating']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].replace('', None)

# 4. Exportar con QUOTE_MINIMAL (solo comillas cuando hay comas/saltos)
df_clean.to_csv('comentarios_clean.csv',
                index=False,
                encoding='utf-8',
                quoting=0,  # QUOTE_MINIMAL - solo cuando sea necesario
                escapechar='\\',
                doublequote=True,
                na_rep='')  # NULL como vacío (sin comillas)

print("✅ Archivo con NULL como vacíos generado")
print(f"📊 Filas: {len(df_clean)}")
print("\n🔍 Línea 107 (debe tener date_response vacío):")
print(df_clean.iloc[106:108][['hotel_name', 'date_posted', 'date_response', 'channel']])

from google.colab import files
files.download('comentarios_clean.csv')

✅ Archivo con NULL como vacíos generado
📊 Filas: 15631

🔍 Línea 107 (debe tener date_response vacío):
                                hotel_name date_posted date_response  \
106  Dreams Vista Cancun Golf & Spa Resort  2021-11-29    2021-11-30   
107    Dreams Bahia Mita Surf & Spa Resort  2021-11-24    2021-11-24   

         channel  
106  TripAdvisor  
107  TripAdvisor  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# PASO 3 ULTRA-FINAL: Escapar comillas correctamente
df_clean = df.copy()

# 1. Limpiar campos de texto
text_columns = ['title_review', 'comment_text', 'key_reason', 'hotel_name', 'channel', 'rating']
for col in text_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str)
        df_clean[col] = df_clean[col].replace('nan', '')
        # Limpiar saltos de línea
        df_clean[col] = df_clean[col].str.replace('\n', ' ', regex=False)
        df_clean[col] = df_clean[col].str.replace('\r', ' ', regex=False)
        # Escapar comillas simples Y dobles
        df_clean[col] = df_clean[col].str.replace("'", "''", regex=False)  # ' -> ''
        df_clean[col] = df_clean[col].str.replace('"', '""', regex=False)  # " -> ""
        df_clean[col] = df_clean[col].replace('', None)

# 2. Convertir fechas
date_columns = ['date_posted', 'date_response']
for col in date_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
        df_clean[col] = df_clean[col].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notna(x) else None)

# 3. Exportar con QUOTE_NONNUMERIC (comillas en todo lo que no sea número)
df_clean.to_csv('comentarios_clean.csv',
                index=False,
                encoding='utf-8',
                quoting=2,  # QUOTE_NONNUMERIC - comillas en strings
                doublequote=True,  # Escapar comillas dobles duplicándolas
                na_rep='')

print("✅ Archivo con comillas escapadas generado")
print(f"📊 Filas: {len(df_clean)}")
print("\n🔍 Línea 15633 (la problemática):")
print(df_clean.iloc[15632:15634])

from google.colab import files
files.download('comentarios_clean.csv')

✅ Archivo con comillas escapadas generado
📊 Filas: 15631

🔍 Línea 15633 (la problemática):
Empty DataFrame
Columns: [hotel_name, date_posted, date_response, channel, rating, title_review, comment_text, key_reason]
Index: []


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# Instalar psycopg2 (driver PostgreSQL para Python)
!pip install psycopg2-binary

import psycopg2
from psycopg2.extras import execute_batch

# Limpiar datos
df_clean = df.copy()

# Limpiar campos de texto
text_columns = ['title_review', 'comment_text', 'key_reason', 'hotel_name', 'channel', 'rating']
for col in text_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).replace('nan', None)
        if col in ['title_review', 'comment_text', 'key_reason']:
            df_clean[col] = df_clean[col].str.replace('\n', ' ', regex=False).str.replace('\r', ' ', regex=False)

# Convertir fechas
for col in ['date_posted', 'date_response']:
    df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

# Reemplazar NaT con None
df_clean = df_clean.where(pd.notna(df_clean), None)

print(f"✅ Datos preparados: {len(df_clean)} filas")
print("\n🔍 Muestra de datos limpios:")
print(df_clean.head(3))

# Conectar a PostgreSQL
conn = psycopg2.connect(
    host="gfg-postgresql-server-2025.postgres.database.azure.com",
    database="gfg_asset_management",
    user="svc_powerbi_reader",
    password="PowerBI2025!",
    sslmode="require"
)

# Preparar datos para inserción
data = [tuple(x) for x in df_clean.to_numpy()]

# SQL de inserción
insert_sql = """
INSERT INTO raw_digital_presence.comentarios
(hotel_name, date_posted, date_response, channel, rating, title_review, comment_text, key_reason)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insertar en batch (más rápido)
cursor = conn.cursor()
execute_batch(cursor, insert_sql, data, page_size=1000)
conn.commit()

print(f"\n✅ {len(data)} filas insertadas exitosamente")

# Cerrar conexión
cursor.close()
conn.close()

print("\n🎉 Migración completada!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.7 MB/s eta 0:00:00
✅ Datos preparados: 15631 filas

🔍 Muestra de datos limpios:
                               hotel_name date_posted date_response  \
0  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
1  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
2  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   

       channel   rating                                       title_review  \
0  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   
1  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   
2  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   

                                        comment_text key_reason  
0  F&B food is okey but the attitude of the staff...        F&B  
1  Room air condition sound is to loud therefore ...       Room  
2     Room bed is very hard and not very comfortable       Room  


InsufficientPrivilege: permission denied for schema raw_digital_presence
LINE 2: INSERT INTO raw_digital_presence.comentarios 
                    ^


In [9]:
# Instalar psycopg2 (driver PostgreSQL para Python)
!pip install psycopg2-binary

import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch

# Asumiendo que 'df' ya existe del paso anterior
# Si no, primero ejecuta: df = pd.read_csv('comentarios.csv', encoding='utf-8', on_bad_lines='skip')

# Limpiar datos
df_clean = df.copy()

# Limpiar campos de texto
text_columns = ['title_review', 'comment_text', 'key_reason', 'hotel_name', 'channel', 'rating']
for col in text_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).replace('nan', None)
        if col in ['title_review', 'comment_text', 'key_reason']:
            df_clean[col] = df_clean[col].str.replace('\n', ' ', regex=False).str.replace('\r', ' ', regex=False)

# Convertir fechas
for col in ['date_posted', 'date_response']:
    df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

# Reemplazar NaT con None (NULL en SQL)
df_clean = df_clean.where(pd.notna(df_clean), None)

print(f"✅ Datos preparados: {len(df_clean)} filas")
print("\n🔍 Muestra de datos limpios:")
print(df_clean.head(3))

# Conectar a PostgreSQL con usuario correcto
conn = psycopg2.connect(
    host="gfg-postgresql-server-2025.postgres.database.azure.com",
    database="gfg_asset_management",
    user="svc_n8n_admin",
    password="PowerBI2025!",
    sslmode="require"
)

# Preparar datos para inserción
data = [tuple(x) for x in df_clean.to_numpy()]

# SQL de inserción
insert_sql = """
INSERT INTO raw_digital_presence.comentarios
(hotel_name, date_posted, date_response, channel, rating, title_review, comment_text, key_reason)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insertar en batch (más rápido)
cursor = conn.cursor()
print("\n⏳ Insertando datos...")
execute_batch(cursor, insert_sql, data, page_size=1000)
conn.commit()

print(f"\n✅ {len(data)} filas insertadas exitosamente en raw_digital_presence.comentarios")

# Cerrar conexión
cursor.close()
conn.close()

print("\n🎉 Migración completada!")

✅ Datos preparados: 15631 filas

🔍 Muestra de datos limpios:
                               hotel_name date_posted date_response  \
0  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
1  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
2  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   

       channel   rating                                       title_review  \
0  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   
1  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   
2  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   

                                        comment_text key_reason  
0  F&B food is okey but the attitude of the staff...        F&B  
1  Room air condition sound is to loud therefore ...       Room  
2     Room bed is very hard and not very comfortable       Room  

⏳ Insertando datos...


InsufficientPrivilege: permission denied for schema raw_digital_presence
LINE 2: INSERT INTO raw_digital_presence.comentarios 
                    ^


In [11]:
# Instalar psycopg2 (driver PostgreSQL para Python)
!pip install psycopg2-binary

import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch

# Asumiendo que 'df' ya existe del paso anterior
# Si no, primero ejecuta: df = pd.read_csv('comentarios.csv', encoding='utf-8', on_bad_lines='skip')

# Limpiar datos
df_clean = df.copy()

# Limpiar campos de texto
text_columns = ['title_review', 'comment_text', 'key_reason', 'hotel_name', 'channel', 'rating']
for col in text_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).replace('nan', None)
        if col in ['title_review', 'comment_text', 'key_reason']:
            df_clean[col] = df_clean[col].str.replace('\n', ' ', regex=False).str.replace('\r', ' ', regex=False)

# Convertir fechas
for col in ['date_posted', 'date_response']:
    df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

# Reemplazar NaT con None (NULL en SQL)
df_clean = df_clean.where(pd.notna(df_clean), None)

print(f"✅ Datos preparados: {len(df_clean)} filas")
print("\n🔍 Muestra de datos limpios:")
print(df_clean.head(3))

# Conectar a PostgreSQL con usuario correcto
conn = psycopg2.connect(
    host="gfg-postgresql-server-2025.postgres.database.azure.com",
    database="gfg_asset_management",
    user="svc_n8n_admin",
    password="PowerBI2025!",
    sslmode="require"
)

# Preparar datos para inserción
data = [tuple(x) for x in df_clean.to_numpy()]

# SQL de inserción
insert_sql = """
INSERT INTO raw_digital_presence.comentarios
(hotel_name, date_posted, date_response, channel, rating, title_review, comment_text, key_reason)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insertar en batch (más rápido)
cursor = conn.cursor()
print("\n⏳ Insertando datos...")
execute_batch(cursor, insert_sql, data, page_size=1000)
conn.commit()

print(f"\n✅ {len(data)} filas insertadas exitosamente en raw_digital_presence.comentarios")

# Cerrar conexión
cursor.close()
conn.close()

print("\n🎉 Migración completada!")

✅ Datos preparados: 15631 filas

🔍 Muestra de datos limpios:
                               hotel_name date_posted date_response  \
0  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
1  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
2  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   

       channel   rating                                       title_review  \
0  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   
1  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   
2  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   

                                        comment_text key_reason  
0  F&B food is okey but the attitude of the staff...        F&B  
1  Room air condition sound is to loud therefore ...       Room  
2     Room bed is very hard and not very comfortable       Room  

⏳ Insertando datos...


InvalidDatetimeFormat: invalid input syntax for type timestamp: "NaT"
LINE 424: ... Resort & Spa', '2021-12-02T00:00:00'::timestamp, 'NaT'::tim...
                                                               ^


In [12]:
# Instalar psycopg2 (driver PostgreSQL para Python)
!pip install psycopg2-binary

import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch
from datetime import datetime

# Limpiar datos
df_clean = df.copy()

# Limpiar campos de texto
text_columns = ['title_review', 'comment_text', 'key_reason', 'hotel_name', 'channel', 'rating']
for col in text_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).replace('nan', None)
        if col in ['title_review', 'comment_text', 'key_reason']:
            df_clean[col] = df_clean[col].str.replace('\n', ' ', regex=False).str.replace('\r', ' ', regex=False)

# Convertir fechas CORRECTAMENTE (sin convertir a string)
for col in ['date_posted', 'date_response']:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
        # Convertir NaT a None ANTES de to_numpy()
        df_clean[col] = df_clean[col].apply(lambda x: x.date() if pd.notna(x) else None)

print(f"✅ Datos preparados: {len(df_clean)} filas")
print("\n🔍 Muestra de datos limpios:")
print(df_clean.head(3))

# Conectar a PostgreSQL
conn = psycopg2.connect(
    host="gfg-postgresql-server-2025.postgres.database.azure.com",
    database="gfg_asset_management",
    user="svc_n8n_admin",
    password="PowerBI2025!",
    sslmode="require"
)

# Preparar datos para inserción
data = [tuple(x) for x in df_clean.to_numpy()]

# SQL de inserción
insert_sql = """
INSERT INTO raw_digital_presence.comentarios
(hotel_name, date_posted, date_response, channel, rating, title_review, comment_text, key_reason)

SyntaxError: incomplete input (ipython-input-890753141.py, line 44)

In [13]:
# Instalar psycopg2
!pip install psycopg2-binary

import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch

# Limpiar datos
df_clean = df.copy()

# Limpiar campos de texto
text_columns = ['title_review', 'comment_text', 'key_reason', 'hotel_name', 'channel', 'rating']
for col in text_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).replace('nan', None)
        if col in ['title_review', 'comment_text', 'key_reason']:
            df_clean[col] = df_clean[col].str.replace('\n', ' ', regex=False).str.replace('\r', ' ', regex=False)

# Convertir fechas CORRECTAMENTE
for col in ['date_posted', 'date_response']:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
        df_clean[col] = df_clean[col].apply(lambda x: x.date() if pd.notna(x) else None)

print(f"Datos preparados: {len(df_clean)} filas")
print("\nMuestra de datos limpios:")
print(df_clean.head(3))

# Conectar a PostgreSQL
conn = psycopg2.connect(
    host="gfg-postgresql-server-2025.postgres.database.azure.com",
    database="gfg_asset_management",
    user="svc_n8n_admin",
    password="PowerBI2025!",
    sslmode="require"
)

# Preparar datos
data = [tuple(x) for x in df_clean.to_numpy()]

# SQL de inserción
insert_sql = """
INSERT INTO raw_digital_presence.comentarios
(hotel_name, date_posted, date_response, channel, rating, title_review, comment_text, key_reason)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insertar en batch
cursor = conn.cursor()
print("\nInsertando datos...")
execute_batch(cursor, insert_sql, data, page_size=1000)
conn.commit()

print(f"\n{len(data)} filas insertadas exitosamente")

# Cerrar conexión
cursor.close()
conn.close()

print("\nMigracion completada!")

Datos preparados: 15631 filas

Muestra de datos limpios:
                               hotel_name date_posted date_response  \
0  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
1  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   
2  Hyatt Place Bogota / Convention Center  2019-11-01    2020-01-12   

       channel   rating                                       title_review  \
0  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   
1  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   
2  TripAdvisor  Average  Atencion del restaurante y calidad de la habit...   

                                        comment_text key_reason  
0  F&B food is okey but the attitude of the staff...        F&B  
1  Room air condition sound is to loud therefore ...       Room  
2     Room bed is very hard and not very comfortable       Room  

Insertando datos...

15631 filas insertadas exitosamente

Migracion completada!
